Actions each agent have to decide between: Leave or Stay 

In [ ]:
import numpy as np

class Agent: 
    def __init__(self, music_taste = 1, strategy = 1):
        self.music_taste = music_taste
        self.position = 0
        self.position_utility = 0
        self.waiting_utility = 0
        self.strategy = strategy
        self.position_history = []

    
class Concert:
    def __init__(self, len_rows = 10, music_type = 1, capacity = 100, duration = 5):
        self.len_rows = len_rows
        self.music_type = music_type
        self.nr_agents = 0
        self.capacity = capacity
        self.nr_rows = np.ceil(capacity/len_rows)
        self.duration = duration #timesteps

    def add_agent(self):
        self.nr_agents += 1

def get_placement_utility(concert):
    placement_factor = 200
    #continuing using sigmoid
    fullness = concert.nr_agents / concert.capacity
    sigmoid_x = 1 - fullness
    k = 10 #steepness of sigomid
    center = 0.5 #[0,1] with 0.1 happy crowd and 0.9 happy front row
    utility = placement_factor * 1 / ( 1 + np.exp(-k*(sigmoid_x- center)))

    #stepwise by which row the agent is in, sensitive to concert row length
    dist_from_stage = np.floor((concert.nr_agents + 1) / concert.len_rows)
    #utility = concert.capacity - (dist_from_stage*concert.nr_rows)
    return utility

def get_time_at_concert_utility(agent):
    utility += agent

def get_waiting_utility(time):
    waiting_factor = 2
    utility = -time * waiting_factor
    return utility

Strategies: Sart with defining strategies that do not depend on any history. Deterministic depending on which timestep it likes to leave

In [9]:
def get_action(agent, timestep, max_timesteps):
    # The one that likes to stay at the consert
    if (agent.strategy == 1):
        if (timestep/max_timesteps <= 0.6):
            action = 'stay'
        else:
            action = 'leave'
        
    # The one that likes to leave to the next consert
    if (agent.strategy == 2):
        if (timestep/max_timesteps >= 0.2):
            action = 'leave'
        else:
            action = 'stay'

    # The lagom one
    if (agent.strategy == 3):
        if (timestep/max_timesteps <= 0.5):
            action = 'leave'
        else:
            action = 'stay'

    return action


Inintialize agents and concerts 

In [ ]:
nr_of_agents = 20
agents = []

nr_conserts = 10
concerts = []

for _ in range (nr_of_agents):
    agents.append(Agent(music_taste=np.random.randint(1,4), strategy=np.random.randint(1,4)))

for _ in range(nr_conserts):
    concerts.append(Concert(len_rows=10, music_type=np.random.randint(1,3), capacity = 5))

#Add all agents to the starting concerts in a random order
shuffled_indices = np.random.permutation(nr_of_agents)
for i in range (nr_of_agents):
    idx = shuffled_indices[i]
    position_utility = 0 #get_placement_utility(concerts[0])
    concerts[0].add_agent() 
    agents[idx].position_utility = position_utility
    agents[idx].position_history.append(position_utility)
    concerts[0].add_agent() 


In [15]:
for i in range(0,len(concerts)-2):  # assume everyone stays at the last concert
    current_concert = concerts[i]
    next_concert = concerts[i + 1]

    counter = 0
    already_left = set()

    for timestep in range(0,current_concert.duration):
        shuffled_indices = np.random.permutation(nr_of_agents)
        for j in range (nr_of_agents):
            idx = shuffled_indices[j]
            if idx in already_left:
                continue

            action = get_action(agents[idx],timestep,current_concert.duration)
            if (action == 'leave'):
                agents[idx].waiting_utility += get_waiting_utility(current_concert.duration - timestep) 
                position_utility = get_placement_utility(next_concert)
                next_concert.add_agent() #TODO: ability to skip a concert
                agents[idx].position_utility = position_utility
                agents[idx].position_history.append(position_utility)
                #TODO: add utility for staying at a concert
                # at the same time, escaping negative waiting utlility is the same

                already_left.add(idx)
                counter += 1
    #print(counter)

How did the simulation go?

In [16]:
sorted_agents = sorted(agents, key=lambda agent: sum(agent.position_history) + agent.waiting_utility, reverse=True)
for i in range (nr_of_agents):
    print("Utility: ", sum(sorted_agents[i].position_history) + sorted_agents[i].waiting_utility, ", Strategy: ", sorted_agents[i].strategy)

Utility:  915.8732907829965 , Strategy:  2
Utility:  634.2736041513649 , Strategy:  2
Utility:  631.6031443503902 , Strategy:  2
Utility:  500.20791528993084 , Strategy:  2
Utility:  440.58908674159886 , Strategy:  2
Utility:  438.0974968951348 , Strategy:  2
Utility:  381.8732478926173 , Strategy:  2
Utility:  336.098715660789 , Strategy:  2
Utility:  -15.998635522009877 , Strategy:  1
Utility:  -15.998643802633989 , Strategy:  1
Utility:  -15.999399593541325 , Strategy:  1
Utility:  -15.999547779895167 , Strategy:  1
Utility:  -15.999808027034863 , Strategy:  1
Utility:  -15.99993545471546 , Strategy:  1
Utility:  -15.99993862401961 , Strategy:  1
Utility:  -15.999938641021785 , Strategy:  1
Utility:  -15.999981972563004 , Strategy:  1
Utility:  -15.999991526997535 , Strategy:  1
Utility:  -15.999996486555432 , Strategy:  1
Utility:  -52.9346662572462 , Strategy:  2


In [ ]:
agents_1 = []
agents_2 = []
agents_3 = []
for agent in agents:
    if agent.strategy == 1 : 
        agents_1.append(agent)
    elif agent.strategy == 2:
        agents_2.append(agent)
    else:
        agents_3.append(agent)

agents_1_utility = [0]
agents_2_utility = [0]
agents_3_utility = [0]

for i in range(0,nr_conserts-1):
    #undone
    sum(agents_1[:].position_history[i])